# Variational Autoencoders Examples

## General Imports

In [3]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import numpy as np

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [11]:
torch.manual_seed(1)

## MNIST

### Load Dataset

In [4]:
# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# transform=transforms.Compose([
#                             transforms.ToTensor(),
#                             transforms.Normalize((0.1307,), (0.3081,))
#                             ])

# choose the training and test datasets
train_data = datasets.MNIST(root='../data', train=True,
                               download=False, transform=transform)
test_data = datasets.MNIST(root='../data', train=False,
                              download=False, transform=transform)

Prepare data loaders

In [5]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 128
# percentage of training set to use as validation
valid_size = 0.2
# epochs
epochs = 10

In [9]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                            num_workers=num_workers)

test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                            num_workers=num_workers)

### Build Model

In [7]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc2_mu = nn.Linear(400, 20)
        self.fc2_logvar = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc2_mu(h1), self.fc2_logvar(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + (std * eps)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        # flatten
        x = x.view(-1, 784)
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

Loss Function and Optimizer

In [8]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

### VAE Loss

In [10]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

### Train Function

In [18]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 1 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

### Test Function

In [144]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

### Training

In [23]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 20).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 117.318672
Train Epoch: 1 [128/60000 (0%)]	Loss: 120.429993
Train Epoch: 1 [256/60000 (0%)]	Loss: 112.470200
Train Epoch: 1 [384/60000 (1%)]	Loss: 120.882759
Train Epoch: 1 [512/60000 (1%)]	Loss: 123.329178
Train Epoch: 1 [640/60000 (1%)]	Loss: 118.879913
Train Epoch: 1 [768/60000 (1%)]	Loss: 118.949081
Train Epoch: 1 [896/60000 (1%)]	Loss: 118.101425
Train Epoch: 1 [1024/60000 (2%)]	Loss: 121.424652
Train Epoch: 1 [1152/60000 (2%)]	Loss: 120.180237
Train Epoch: 1 [1280/60000 (2%)]	Loss: 118.222214
Train Epoch: 1 [1408/60000 (2%)]	Loss: 120.635941
Train Epoch: 1 [1536/60000 (3%)]	Loss: 119.455856
Train Epoch: 1 [1664/60000 (3%)]	Loss: 116.220917
Train Epoch: 1 [1792/60000 (3%)]	Loss: 114.788185
Train Epoch: 1 [1920/60000 (3%)]	Loss: 115.538483
Train Epoch: 1 [2048/60000 (3%)]	Loss: 116.328163
Train Epoch: 1 [2176/60000 (4%)]	Loss: 121.571930
Train Epoch: 1 [2304/60000 (4%)]	Loss: 117.991096
Train Epoch: 1 [2432/60000 (4%)]	Loss: 118.785034
Train Epoc

Train Epoch: 1 [21760/60000 (36%)]	Loss: 116.261963
Train Epoch: 1 [21888/60000 (36%)]	Loss: 113.856430
Train Epoch: 1 [22016/60000 (37%)]	Loss: 130.138016
Train Epoch: 1 [22144/60000 (37%)]	Loss: 125.832100
Train Epoch: 1 [22272/60000 (37%)]	Loss: 117.735909
Train Epoch: 1 [22400/60000 (37%)]	Loss: 118.200485
Train Epoch: 1 [22528/60000 (38%)]	Loss: 117.736313
Train Epoch: 1 [22656/60000 (38%)]	Loss: 121.039948
Train Epoch: 1 [22784/60000 (38%)]	Loss: 113.866455
Train Epoch: 1 [22912/60000 (38%)]	Loss: 119.970703
Train Epoch: 1 [23040/60000 (38%)]	Loss: 119.974106
Train Epoch: 1 [23168/60000 (39%)]	Loss: 110.502716
Train Epoch: 1 [23296/60000 (39%)]	Loss: 117.104263
Train Epoch: 1 [23424/60000 (39%)]	Loss: 113.826752
Train Epoch: 1 [23552/60000 (39%)]	Loss: 121.473648
Train Epoch: 1 [23680/60000 (39%)]	Loss: 124.299171
Train Epoch: 1 [23808/60000 (40%)]	Loss: 125.211784
Train Epoch: 1 [23936/60000 (40%)]	Loss: 113.359406
Train Epoch: 1 [24064/60000 (40%)]	Loss: 109.704666
Train Epoch:

Train Epoch: 1 [43392/60000 (72%)]	Loss: 117.198532
Train Epoch: 1 [43520/60000 (72%)]	Loss: 117.064636
Train Epoch: 1 [43648/60000 (73%)]	Loss: 110.202400
Train Epoch: 1 [43776/60000 (73%)]	Loss: 116.888123
Train Epoch: 1 [43904/60000 (73%)]	Loss: 112.468231
Train Epoch: 1 [44032/60000 (73%)]	Loss: 121.739342
Train Epoch: 1 [44160/60000 (74%)]	Loss: 122.110016
Train Epoch: 1 [44288/60000 (74%)]	Loss: 116.436485
Train Epoch: 1 [44416/60000 (74%)]	Loss: 116.232376
Train Epoch: 1 [44544/60000 (74%)]	Loss: 111.381729
Train Epoch: 1 [44672/60000 (74%)]	Loss: 119.897125
Train Epoch: 1 [44800/60000 (75%)]	Loss: 118.036598
Train Epoch: 1 [44928/60000 (75%)]	Loss: 114.293922
Train Epoch: 1 [45056/60000 (75%)]	Loss: 116.338028
Train Epoch: 1 [45184/60000 (75%)]	Loss: 118.204849
Train Epoch: 1 [45312/60000 (75%)]	Loss: 117.009460
Train Epoch: 1 [45440/60000 (76%)]	Loss: 120.616379
Train Epoch: 1 [45568/60000 (76%)]	Loss: 115.449997
Train Epoch: 1 [45696/60000 (76%)]	Loss: 118.322388
Train Epoch:

Train Epoch: 2 [4992/60000 (8%)]	Loss: 112.822159
Train Epoch: 2 [5120/60000 (9%)]	Loss: 115.761513
Train Epoch: 2 [5248/60000 (9%)]	Loss: 118.906982
Train Epoch: 2 [5376/60000 (9%)]	Loss: 112.835098
Train Epoch: 2 [5504/60000 (9%)]	Loss: 113.895996
Train Epoch: 2 [5632/60000 (9%)]	Loss: 114.862457
Train Epoch: 2 [5760/60000 (10%)]	Loss: 110.862106
Train Epoch: 2 [5888/60000 (10%)]	Loss: 111.196991
Train Epoch: 2 [6016/60000 (10%)]	Loss: 110.906334
Train Epoch: 2 [6144/60000 (10%)]	Loss: 109.923683
Train Epoch: 2 [6272/60000 (10%)]	Loss: 109.853424
Train Epoch: 2 [6400/60000 (11%)]	Loss: 112.801247
Train Epoch: 2 [6528/60000 (11%)]	Loss: 106.430862
Train Epoch: 2 [6656/60000 (11%)]	Loss: 109.525589
Train Epoch: 2 [6784/60000 (11%)]	Loss: 116.163162
Train Epoch: 2 [6912/60000 (12%)]	Loss: 113.195740
Train Epoch: 2 [7040/60000 (12%)]	Loss: 111.177437
Train Epoch: 2 [7168/60000 (12%)]	Loss: 118.117661
Train Epoch: 2 [7296/60000 (12%)]	Loss: 113.092102
Train Epoch: 2 [7424/60000 (12%)]	Los

Train Epoch: 2 [26496/60000 (44%)]	Loss: 113.798279
Train Epoch: 2 [26624/60000 (44%)]	Loss: 114.939819
Train Epoch: 2 [26752/60000 (45%)]	Loss: 110.399155
Train Epoch: 2 [26880/60000 (45%)]	Loss: 109.328835
Train Epoch: 2 [27008/60000 (45%)]	Loss: 119.690765
Train Epoch: 2 [27136/60000 (45%)]	Loss: 122.163628
Train Epoch: 2 [27264/60000 (45%)]	Loss: 116.635101
Train Epoch: 2 [27392/60000 (46%)]	Loss: 118.531937
Train Epoch: 2 [27520/60000 (46%)]	Loss: 108.387604
Train Epoch: 2 [27648/60000 (46%)]	Loss: 113.706757
Train Epoch: 2 [27776/60000 (46%)]	Loss: 116.499046
Train Epoch: 2 [27904/60000 (46%)]	Loss: 108.205215
Train Epoch: 2 [28032/60000 (47%)]	Loss: 106.572205
Train Epoch: 2 [28160/60000 (47%)]	Loss: 114.437759
Train Epoch: 2 [28288/60000 (47%)]	Loss: 111.408379
Train Epoch: 2 [28416/60000 (47%)]	Loss: 111.163193
Train Epoch: 2 [28544/60000 (48%)]	Loss: 110.168587
Train Epoch: 2 [28672/60000 (48%)]	Loss: 112.414032
Train Epoch: 2 [28800/60000 (48%)]	Loss: 115.017059
Train Epoch:

...

Train Epoch: 9 [16000/60000 (27%)]	Loss: 104.048767
Train Epoch: 9 [16128/60000 (27%)]	Loss: 102.486916
Train Epoch: 9 [16256/60000 (27%)]	Loss: 103.479797
Train Epoch: 9 [16384/60000 (27%)]	Loss: 102.277939
Train Epoch: 9 [16512/60000 (28%)]	Loss: 107.795013
Train Epoch: 9 [16640/60000 (28%)]	Loss: 103.053589
Train Epoch: 9 [16768/60000 (28%)]	Loss: 116.184586
Train Epoch: 9 [16896/60000 (28%)]	Loss: 102.640297
Train Epoch: 9 [17024/60000 (28%)]	Loss: 106.466942
Train Epoch: 9 [17152/60000 (29%)]	Loss: 112.929298
Train Epoch: 9 [17280/60000 (29%)]	Loss: 109.831451
Train Epoch: 9 [17408/60000 (29%)]	Loss: 107.887489
Train Epoch: 9 [17536/60000 (29%)]	Loss: 115.833809
Train Epoch: 9 [17664/60000 (29%)]	Loss: 103.551147
Train Epoch: 9 [17792/60000 (30%)]	Loss: 100.593559
Train Epoch: 9 [17920/60000 (30%)]	Loss: 103.370941
Train Epoch: 9 [18048/60000 (30%)]	Loss: 100.107185
Train Epoch: 9 [18176/60000 (30%)]	Loss: 99.147514
Train Epoch: 9 [18304/60000 (30%)]	Loss: 100.821686
Train Epoch: 

Train Epoch: 9 [37248/60000 (62%)]	Loss: 111.791496
Train Epoch: 9 [37376/60000 (62%)]	Loss: 109.564072
Train Epoch: 9 [37504/60000 (62%)]	Loss: 110.024658
Train Epoch: 9 [37632/60000 (63%)]	Loss: 109.035072
Train Epoch: 9 [37760/60000 (63%)]	Loss: 110.881424
Train Epoch: 9 [37888/60000 (63%)]	Loss: 103.836662
Train Epoch: 9 [38016/60000 (63%)]	Loss: 107.298538
Train Epoch: 9 [38144/60000 (64%)]	Loss: 110.077332
Train Epoch: 9 [38272/60000 (64%)]	Loss: 109.669556
Train Epoch: 9 [38400/60000 (64%)]	Loss: 102.226997
Train Epoch: 9 [38528/60000 (64%)]	Loss: 111.137764
Train Epoch: 9 [38656/60000 (64%)]	Loss: 106.300079
Train Epoch: 9 [38784/60000 (65%)]	Loss: 106.987595
Train Epoch: 9 [38912/60000 (65%)]	Loss: 104.557053
Train Epoch: 9 [39040/60000 (65%)]	Loss: 101.640686
Train Epoch: 9 [39168/60000 (65%)]	Loss: 103.125038
Train Epoch: 9 [39296/60000 (65%)]	Loss: 106.857544
Train Epoch: 9 [39424/60000 (66%)]	Loss: 102.743355
Train Epoch: 9 [39552/60000 (66%)]	Loss: 101.956848
Train Epoch:

Train Epoch: 9 [59776/60000 (100%)]	Loss: 103.384460
Train Epoch: 9 [44928/60000 (100%)]	Loss: 108.948832
====> Epoch: 9 Average loss: 106.4710
====> Test set loss: 106.6400
Train Epoch: 10 [0/60000 (0%)]	Loss: 105.139717
Train Epoch: 10 [128/60000 (0%)]	Loss: 105.849609
Train Epoch: 10 [256/60000 (0%)]	Loss: 99.783333
Train Epoch: 10 [384/60000 (1%)]	Loss: 105.822800
Train Epoch: 10 [512/60000 (1%)]	Loss: 108.843452
Train Epoch: 10 [640/60000 (1%)]	Loss: 106.792572
Train Epoch: 10 [768/60000 (1%)]	Loss: 105.573242
Train Epoch: 10 [896/60000 (1%)]	Loss: 104.930717
Train Epoch: 10 [1024/60000 (2%)]	Loss: 109.376373
Train Epoch: 10 [1152/60000 (2%)]	Loss: 108.225388
Train Epoch: 10 [1280/60000 (2%)]	Loss: 104.697609
Train Epoch: 10 [1408/60000 (2%)]	Loss: 106.272690
Train Epoch: 10 [1536/60000 (3%)]	Loss: 106.149307
Train Epoch: 10 [1664/60000 (3%)]	Loss: 104.055054
Train Epoch: 10 [1792/60000 (3%)]	Loss: 101.989067
Train Epoch: 10 [1920/60000 (3%)]	Loss: 103.172607
Train Epoch: 10 [2048

Train Epoch: 10 [22272/60000 (37%)]	Loss: 106.328453
Train Epoch: 10 [22400/60000 (37%)]	Loss: 106.954903
Train Epoch: 10 [22528/60000 (38%)]	Loss: 105.223145
Train Epoch: 10 [22656/60000 (38%)]	Loss: 109.060402
Train Epoch: 10 [22784/60000 (38%)]	Loss: 103.336578
Train Epoch: 10 [22912/60000 (38%)]	Loss: 108.482933
Train Epoch: 10 [23040/60000 (38%)]	Loss: 108.233948
Train Epoch: 10 [23168/60000 (39%)]	Loss: 100.388443
Train Epoch: 10 [23296/60000 (39%)]	Loss: 105.946251
Train Epoch: 10 [23424/60000 (39%)]	Loss: 103.065002
Train Epoch: 10 [23552/60000 (39%)]	Loss: 108.842575
Train Epoch: 10 [23680/60000 (39%)]	Loss: 111.928200
Train Epoch: 10 [23808/60000 (40%)]	Loss: 112.883240
Train Epoch: 10 [23936/60000 (40%)]	Loss: 104.568550
Train Epoch: 10 [24064/60000 (40%)]	Loss: 100.687057
Train Epoch: 10 [24192/60000 (40%)]	Loss: 103.367981
Train Epoch: 10 [24320/60000 (41%)]	Loss: 102.601685
Train Epoch: 10 [24448/60000 (41%)]	Loss: 107.396492
Train Epoch: 10 [24576/60000 (41%)]	Loss: 108.

Train Epoch: 10 [44800/60000 (75%)]	Loss: 108.780579
Train Epoch: 10 [44928/60000 (75%)]	Loss: 104.649719
Train Epoch: 10 [45056/60000 (75%)]	Loss: 106.727760
Train Epoch: 10 [45184/60000 (75%)]	Loss: 107.680687
Train Epoch: 10 [45312/60000 (75%)]	Loss: 107.817436
Train Epoch: 10 [45440/60000 (76%)]	Loss: 110.962189
Train Epoch: 10 [45568/60000 (76%)]	Loss: 104.859436
Train Epoch: 10 [45696/60000 (76%)]	Loss: 109.923439
Train Epoch: 10 [45824/60000 (76%)]	Loss: 110.586060
Train Epoch: 10 [45952/60000 (77%)]	Loss: 109.514725
Train Epoch: 10 [46080/60000 (77%)]	Loss: 104.759872
Train Epoch: 10 [46208/60000 (77%)]	Loss: 109.979828
Train Epoch: 10 [46336/60000 (77%)]	Loss: 109.663437
Train Epoch: 10 [46464/60000 (77%)]	Loss: 107.308418
Train Epoch: 10 [46592/60000 (78%)]	Loss: 109.957642
Train Epoch: 10 [46720/60000 (78%)]	Loss: 106.677856
Train Epoch: 10 [46848/60000 (78%)]	Loss: 106.172768
Train Epoch: 10 [46976/60000 (78%)]	Loss: 105.292625
Train Epoch: 10 [47104/60000 (78%)]	Loss: 106.

## FRUITS

In [2]:
import torch
import os
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
# number of subprocesses to use for data loading
num_workers = 2
# how many samples per batch to load
batch_size = 128
# percentage of training set to use as validation
# valid_size = 0.2
# epochs
epochs = 30

log_interval = 50

### Data Loaders

Image size is  100 x 100

In [7]:
root_train = "../data/fruits-360/Training"
root_test = "../data/fruits-360/Test"

In [8]:
train_loader = torch.utils.data.DataLoader(datasets.ImageFolder(
    root_train, transform=transforms.ToTensor()),
                                           batch_size=batch_size,
                                           num_workers=num_workers,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(datasets.ImageFolder(
    root_test, transform=transforms.ToTensor()),
                                          batch_size=batch_size,
                                          num_workers=num_workers,
                                          shuffle=True)

In [9]:
train_loader.batch_size

128

In [10]:
len(train_loader)

529

In [11]:
len(train_loader.sampler)

67692

In [12]:
dataiter = iter(train_loader)
batch_data = dataiter.next()

In [13]:
len(batch_data)

2

In [14]:
batch_data[0].shape

torch.Size([128, 3, 100, 100])

In [15]:
batch_data[1].shape

torch.Size([128])

In [16]:
batch_data[1]


tensor([101, 108,  84,  97,  73,  85, 109, 129,  67,  92,   8, 123, 123,  73,
         63,  58,  97,  28,  80,  29,  44, 104,  28,  98,  91, 119,  27,  17,
         94,  76,  46, 104, 107,  55,  23,   7, 112,  21, 126,  57,  73,  68,
         78,  59,  72,  92,  19,  77,  58, 128, 104,  20, 122,  24, 118,  36,
         51,  35, 104, 117,  79, 107, 112,  45,  34,  31,  34, 120,  25,  16,
         57,   6,  85,  41,  38, 125,  61, 122,  70,  16,  21,  59,  32,  76,
        105, 128,   2,  28,  52,  29,  69,   8,  89,  67, 107,  99,  67, 111,
         96, 110,  78,  92,  27,  69,  28,  42,  52,  50, 110,  67,  76,   6,
         66,  60,  76, 129,  29,  17,  59,  22, 123,  73,  79,  30,  41,  79,
         68,  68])

### Build Model

<img src="img/VAE_Image_1.png" width="400" height="200">

In [43]:
class VAE_CNN_Fruits(nn.Module):
    def __init__(self):
        super().__init__()

        # Encoder
        self.conv1 = nn.Conv2d(3, 16, kernel_size=4, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.conv3 = nn.Conv2d(32, 16, kernel_size=5, stride=2)

        # Latent vectors mu and sigma
        self.fc1 = nn.Linear(10 * 10 * 16, 1024)
        self.fc21 = nn.Linear(1024, 1024)
        self.fc22 = nn.Linear(1024, 1024)

        # Sampling vector
        self.fc3 = nn.Linear(1024, 1024)
        self.fc4 = nn.Linear(1024, 10 * 10 * 16)

        # Decoder
        self.convT1 = nn.ConvTranspose2d(16, 32, kernel_size=5, stride=2)
        self.convT2 = nn.ConvTranspose2d(32, 16, kernel_size=5, stride=2)
        self.convT3 = nn.ConvTranspose2d(16, 3, kernel_size=4, stride=2) 
        
    def encoder(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        
        x = self.conv2(x)
        x = F.relu(x)

        x = self.conv3(x)
        x = F.relu(x)
        
        x = x.view(-1, 10 * 10 * 16)
        
        x = self.fc1(x)
        x = F.relu(x)
        
        mu = self.fc21(x)
        logvar = self.fc22(x)
        
        return mu, logvar

    
    def reparameterize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        eps = std.data.new(std.size()).normal_()
        return eps.mul(std).add_(mu)


    def decoder(self, z):
        x = self.fc3(z)
        x = F.relu(x)
        
        x = self.fc4(x)
        x = F.relu(x)
        
        x = x.view(-1, 16, 10, 10)
        
        x = self.convT1(x)
        x = F.relu(x)
        
        x = self.convT2(x)
        x = F.relu(x)
        
        x = self.convT3(x)
        x = F.relu(x)

        return x

    
    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

In [51]:
class VAE_Loss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse_loss = nn.MSELoss(reduction="sum")

    def forward(self, x_recon, x, mu, logvar):
        loss_MSE = self.mse_loss(x_recon, x)
        loss_KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return loss_MSE + loss_KLD
    
# def VAE_Loss(recon_x, x, mu, logvar):
#     BCE = F.binary_cross_entropy(recon_x, x, reduction="sum")
#     # BCE = F.mse_loss(recon_x, x, size_average=False)

#     # see Appendix B from VAE paper:
#     # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
#     # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
#     KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

#     return BCE + KLD, BCE, KLD

In [52]:
model = VAE_CNN_Fruits().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [53]:
print(model)

VAE_CNN_Fruits(
  (conv1): Conv2d(3, 16, kernel_size=(4, 4), stride=(2, 2))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2))
  (conv3): Conv2d(32, 16, kernel_size=(5, 5), stride=(2, 2))
  (fc1): Linear(in_features=1600, out_features=1024, bias=True)
  (fc21): Linear(in_features=1024, out_features=1024, bias=True)
  (fc22): Linear(in_features=1024, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=1600, bias=True)
  (convT1): ConvTranspose2d(16, 32, kernel_size=(5, 5), stride=(2, 2))
  (convT2): ConvTranspose2d(32, 16, kernel_size=(5, 5), stride=(2, 2))
  (convT3): ConvTranspose2d(16, 3, kernel_size=(4, 4), stride=(2, 2))
)


In [54]:
VAE_Loss = VAE_Loss()

###  Train

In [55]:
val_losses = []
train_losses = []

In [56]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = VAE_Loss(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                       loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))
    train_losses.append(train_loss / len(train_loader.dataset))

In [57]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += VAE_Loss(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                        recon_batch.view(batch_size, 3, 100, 100)[:n]])
                save_image(comparison.cpu(),
                           'results/Fruit_reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))
    val_losses.append(test_loss)

In [58]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 1024).to(device)
        sample = model.decoder(sample).cpu()
        save_image(sample.view(64, 3, 100, 100),
                   'results/Fruit_sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/67692 (0%)]	Loss: 12850.348633
Train Epoch: 1 [6400/67692 (9%)]	Loss: 2611.060547
Train Epoch: 1 [12800/67692 (19%)]	Loss: 1797.346436
Train Epoch: 1 [19200/67692 (28%)]	Loss: 1433.793945
Train Epoch: 1 [25600/67692 (38%)]	Loss: 1189.395508
Train Epoch: 1 [32000/67692 (47%)]	Loss: 1067.505493
Train Epoch: 1 [38400/67692 (57%)]	Loss: 1007.063416
Train Epoch: 1 [44800/67692 (66%)]	Loss: 929.710815
Train Epoch: 1 [51200/67692 (76%)]	Loss: 919.563904
Train Epoch: 1 [57600/67692 (85%)]	Loss: 809.668823
Train Epoch: 1 [64000/67692 (95%)]	Loss: 778.427307
====> Epoch: 1 Average loss: 1584.7651
====> Test set loss: 754.5203
Train Epoch: 2 [0/67692 (0%)]	Loss: 756.217834
Train Epoch: 2 [6400/67692 (9%)]	Loss: 762.467773
Train Epoch: 2 [12800/67692 (19%)]	Loss: 681.513916
Train Epoch: 2 [19200/67692 (28%)]	Loss: 692.843750
Train Epoch: 2 [25600/67692 (38%)]	Loss: 621.243164
Train Epoch: 2 [32000/67692 (47%)]	Loss: 616.835083
Train Epoch: 2 [38400/67692 (57%)]	Loss: 648.037354
T

====> Epoch: 13 Average loss: 255.9971
====> Test set loss: 268.4610
Train Epoch: 14 [0/67692 (0%)]	Loss: 241.594147
Train Epoch: 14 [6400/67692 (9%)]	Loss: 262.140839
Train Epoch: 14 [12800/67692 (19%)]	Loss: 254.913300
Train Epoch: 14 [19200/67692 (28%)]	Loss: 246.283081
Train Epoch: 14 [25600/67692 (38%)]	Loss: 246.913879
Train Epoch: 14 [32000/67692 (47%)]	Loss: 265.698944
Train Epoch: 14 [38400/67692 (57%)]	Loss: 257.107330
Train Epoch: 14 [44800/67692 (66%)]	Loss: 250.062225
Train Epoch: 14 [51200/67692 (76%)]	Loss: 239.639084
Train Epoch: 14 [57600/67692 (85%)]	Loss: 236.400604
Train Epoch: 14 [64000/67692 (95%)]	Loss: 262.122864
====> Epoch: 14 Average loss: 249.7846
====> Test set loss: 269.5839
Train Epoch: 15 [0/67692 (0%)]	Loss: 261.938416
Train Epoch: 15 [6400/67692 (9%)]	Loss: 249.635010
Train Epoch: 15 [12800/67692 (19%)]	Loss: 268.294067
Train Epoch: 15 [19200/67692 (28%)]	Loss: 243.227020
Train Epoch: 15 [25600/67692 (38%)]	Loss: 254.868225
Train Epoch: 15 [32000/67692

Train Epoch: 26 [51200/67692 (76%)]	Loss: 219.082779
Train Epoch: 26 [57600/67692 (85%)]	Loss: 217.853775
Train Epoch: 26 [64000/67692 (95%)]	Loss: 213.570343
====> Epoch: 26 Average loss: 216.6648
====> Test set loss: 243.3757
Train Epoch: 27 [0/67692 (0%)]	Loss: 214.436874
Train Epoch: 27 [6400/67692 (9%)]	Loss: 203.039169
Train Epoch: 27 [12800/67692 (19%)]	Loss: 209.949417
Train Epoch: 27 [19200/67692 (28%)]	Loss: 212.925140
Train Epoch: 27 [25600/67692 (38%)]	Loss: 216.833755
Train Epoch: 27 [32000/67692 (47%)]	Loss: 223.006439
Train Epoch: 27 [38400/67692 (57%)]	Loss: 215.342911
Train Epoch: 27 [44800/67692 (66%)]	Loss: 214.902496
Train Epoch: 27 [51200/67692 (76%)]	Loss: 204.444489
Train Epoch: 27 [57600/67692 (85%)]	Loss: 212.990021
Train Epoch: 27 [64000/67692 (95%)]	Loss: 204.368011
====> Epoch: 27 Average loss: 215.2522
====> Test set loss: 242.1285
Train Epoch: 28 [0/67692 (0%)]	Loss: 211.320145
Train Epoch: 28 [6400/67692 (9%)]	Loss: 200.095734
Train Epoch: 28 [12800/67692

## Helper Functions

In [1]:
def conv2d_dimensions(width_in, height_in, input_channels, kernel_size, kernel_filters, padding=0, stride=1):

    width_out = ((width_in - kernel_size + (2 * padding)) / stride) + 1
    height_out = ((height_in - kernel_size + (2 * padding)) / stride) + 1
    output_channels = kernel_filters
    
    return width_out, height_out, output_channels

In [2]:
def convTranspose2d_dim(width_in, height_in, input_channels, kernel_size, kernel_filters, padding=0, stride=1):
    output_channels = kernel_filters
    width_out = stride * (width_in - 1) + kernel_size - (2 * padding)
    height_out = stride * (height_in - 1) + kernel_size - (2 * padding)
    
    return width_out, height_out, output_channels

## Helper Calculations

### conv2d

In [10]:
conv2d_dimensions(100, 100, 3, 3, 16)

(98.0, 98.0, 16)

In [22]:
conv2d_dimensions(98, 98, 16, 3, 32)

(96.0, 96.0, 32)

In [23]:
conv2d_dimensions(96/2, 96/2, 32, 3, 16, padding=0, stride=1)

(46.0, 46.0, 16)

Conv2D with stride=2 for downsampling

In [34]:
conv2d_dimensions(100, 100, 3, 4, 16, padding=0, stride=2)

(49.0, 49.0, 16)

In [31]:
conv2d_dimensions(49, 49, 16, 5, 32, padding=0, stride=2)

(23.0, 23.0, 32)

In [32]:
conv2d_dimensions(23, 23, 32, 5, 16, padding=0, stride=2)

(10.0, 10.0, 16)

### ConvTranspose2D

ConvTranspose2D Dimensions calculation

In [72]:
convTranspose2d_dim(23, 23, 16, 16, 32)

(38, 38, 32)

In [73]:
convTranspose2d_dim(38, 38, 32, 30, 16)

(67, 67, 16)

In [74]:
convTranspose2d_dim(67, 67, 16, 34, 3)

(100, 100, 3)

Conv2DTranspose with stride=2

In [40]:
convTranspose2d_dim(10, 10, 16, 5, 32, padding=0, stride=2)

(23, 23, 32)

### Flatten

In [21]:
t = torch.tensor([[[1, 2],
                       [3, 4]],
                      [[5, 6],
                       [7, 8]]])
t

tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])

In [22]:
torch.flatten(t)

tensor([1, 2, 3, 4, 5, 6, 7, 8])

In [23]:
torch.flatten(t, 1)

tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]])

In [38]:
convTranspose2d_dim(23, 23, 32, 5, 16, padding=0, stride=2)

(49, 49, 16)

In [39]:
convTranspose2d_dim(49, 49, 16, 4, 3, padding=0, stride=2)

(100, 100, 3)

### Test BCE Loss values

INPUTS

In [22]:
input = torch.randn((2, 1), requires_grad=True)
input

tensor([[-0.9193],
        [ 0.9176]], requires_grad=True)

TARGETS

In [35]:
target = torch.rand((2, 1), requires_grad=False)
target

tensor([[0.2970],
        [0.4710]])

SIGMOIDS TO INPUTS

In [39]:
input_sig = torch.sigmoid(input)
input_sig

tensor([[0.2851],
        [0.7145]], grad_fn=<SigmoidBackward>)

In [49]:
loss = F.binary_cross_entropy(torch.sigmoid(input), target, reduction='mean')
loss

tensor(0.7151, grad_fn=<BinaryCrossEntropyBackward>)

In [46]:
-( (0.2970 * np.log(0.2851)) + (1 - 0.2970)*(np.log(1 - 0.2851)))

0.6086455012350224

In [47]:
-( (0.4710 * np.log(0.7145)) + (1 - 0.4710)*(np.log(1 - 0.7145)))

0.8214456538290854

In [50]:
(0.6086455012350224+0.8214456538290854)/2

0.7150455775320539